In [30]:
import mujoco
from mujoco import MjModel, MjData
from mujoco.viewer import launch_passive
from mujoco import viewer
import time

model = mujoco.MjModel.from_xml_path("tesbot.urdf")
data = mujoco.MjData(model)

In [31]:
with viewer.launch_passive(model, data) as viewer:
    for _ in range(100):
        mujoco.mj_step(model, data)
        viewer.sync()  # Ensure the viewer updates each frame
        time.sleep(model.opt.timestep)  # Real-time step duration
        

In [39]:
import os
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import mujoco


class QuadrupedEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 60}

    def __init__(self, render_mode=None):
        super().__init__()
        
        # Load the URDF model
        model_path = "tesbot.urdf"
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Missing URDF: {model_path}")

        # Convert URDF to MJCF model
        self.model = mujoco.MjModel.from_xml_path(model_path)
        self.data = mujoco.MjData(self.model)

        # Action and observation space (8 joints)
        self.n_actuators = self.model.nu
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(self.n_actuators,), dtype=np.float32)

        obs_size = self.model.nq + self.model.nv
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float32)

        self.render_mode = render_mode
        self.viewer = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        # Reset state
        self.data.qpos[:] = 0
        self.data.qvel[:] = 0
        mujoco.mj_forward(self.model, self.data)

        observation = self._get_obs()
        return observation, {}

    def step(self, action):
        # Clip to action limits
        action = np.clip(action, self.action_space.low, self.action_space.high)
        self.data.ctrl[:] = action

        mujoco.mj_step(self.model, self.data)

        obs = self._get_obs()

        # Basic reward for staying upright
        reward = 1.0 if self.data.qpos[2] > 0.05 else 0.0
        terminated = False
        truncated = False

        return obs, reward, terminated, truncated, {}

    def _get_obs(self):
        return np.concatenate([self.data.qpos, self.data.qvel]).copy()

    def render(self):
        if self.render_mode == "human":
            if self.viewer is None:
                self.viewer = mujoco.viewer.launch_passive(self.model, self.data)
            self.viewer.sync()

    def close(self):
        if self.viewer:
            self.viewer.close()
            self.viewer = None
env = QuadrupedEnv(render_mode="human")
env.reset()

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), {})